In [1]:
import os
os.environ['PATH']+=':/usr/local/cuda/bin/' # cuda must be in path
os.environ['CUDA_PATH']='/usr/local/cuda'

import idi.simulation as sim
import numpy as np
from numpy import pi
!hostname
import matplotlib.pyplot as plt
%matplotlib inline

psanagpu112


In [2]:
!where nvcc

/usr/local/cuda/bin/nvcc
/usr/local/cuda/bin/nvcc
/usr/local/cuda/bin//nvcc


In [2]:
import mkl
mkl.domain_set_num_threads(8,'vml')
mkl.set_num_threads(8)

16

**Settings**

In [3]:
Natoms = int(1e5)
Ndet = 4096
detz = 14 # in cm
pixelsize = 25 # in um
Nimg = 100
E = 9200  # in ev
rotangles = np.array([0,0,0]) / 180 * pi
cuda = True
r = 5 #nm

1

**Simulation**

In [4]:
print("preparing")
_detz = detz * 1e4 # in um
k = 2 * pi / (1.24 / E)  # in 1/um
N = Natoms
_r = r * 1e-3 # in um
_a = 5.6*1e-4
repeats=[200,200,1000]
simobject = sim.simobj.fcc(E,N,_a,repeats=repeats)
simobject.rndPhase=True
simobject.rndPos=False
c=0.3 #um/fs
tau=.1 #fs

preparing


In [12]:
r.dtype

dtype('complex128')

In [ ]:
r1=sim.cuda.simulate(Nimg,simobject,Ndet,pixelsize,_detz,k)
r1=np.abs(r1)**2

0.. 1.. 2.. 3.. 4.. 5.. 6.. 7.. 8.. 9.. 10.. 11.. 12.. 13.. 14.

In [ ]:
np.savez_compressed('/reg/data/ana15/xpp/xppx37817/scratch/zimmf/msc/fcc_temp2_1.npz',r=r1,detz=_detz,pixelsize=pixelsize,a=_a,Natoms=Natoms,repeats=repeats,E=E,k=k)

In [ ]:
r2=sim.cuda.simulate(Nimg,simobject,Ndet,pixelsize,_detz,k)
r2=np.abs(r2)**2


In [ ]:
np.savez_compressed('/reg/data/ana15/xpp/xppx37817/scratch/zimmf/msc/fcc_temp2.npz',r=r2,detz=_detz,pixelsize=pixelsize,a=_a,Natoms=Natoms,repeats=repeats,E=E,k=k)

In [ ]:
import idi.reconstruction as recon

In [51]:
import idi.util as util
r=util.rebin(np.concatenate((r1,r2)),[1,2,2])

In [9]:
import mkl
mkl.domain_set_num_threads(8,'fft')

'success'

In [30]:
corr=recon.ft.corr(r,_detz/(2*pixelsize),True)/len(r)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

In [34]:
corr=recon.ft.unwrap(corr)

In [35]:
norm=recon.ft.unwrap(recon.ft.corr(np.mean(r,0),_detz/(2*pixelsize),True))

. 

In [ ]:
import gc
gc.collect()

In [36]:
t=corr[0]/norm[0]
t[norm[0]<1e-20]=np.nan

<ipython-input-36-b02314c5cb7f>:1: RuntimeWarning: divide by zero encountered in true_divide
  t=corr[0]/norm[0]
<ipython-input-36-b02314c5cb7f>:1: RuntimeWarning: invalid value encountered in true_divide
  t=corr[0]/norm[0]


In [24]:
np.nanmin(t)

0.02085183474140936

In [ ]:
plt.rcParams['figure.figsize']=20,20
plt.matshow(t-1,vmin=0,vmax=.01)
plt.colorbar()

In [ ]:
plt.rcParams['figure.figsize']=20,20
plt.matshow(t-1,vmin=0,vmax=.01)
plt.colorbar()